# Loading a DSE Vector




## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [ ]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


## Useful Functions

In [ ]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

## Load a Template File

Load it:

In [ ]:
val filePath = s"aesop.txt"
val allLines: Vector[String] = {
    scala.io.Source.fromFile(filePath).mkString.split("\n").toVector.filter( _.size > 0 )
}

We define a custom Class that is String + Index:

In [ ]:
case class IndexedLine( text: String, index: Int)

We want to separate heading-lines from the content-lines.

In [ ]:
val pattern = "^[0-9]".r

val indexedLines: Vector[IndexedLine] = allLines.zipWithIndex.map ( l => {
    IndexedLine( l._1, l._2 )
})

val headingLines: Vector[IndexedLine] = indexedLines.filter( l => {
    pattern.findAllIn(l.text).size > 0
})

In [ ]:
val headingPairs: Vector[Vector[IndexedLine]] = headingLines.sliding(2,1).toVector
val mappedHeadings: Vector[( IndexedLine, Vector[IndexedLine])] = {
    val allButLast: Vector[( IndexedLine, Vector[IndexedLine])] = headingPairs.map( p => {
        val firstIndex: Int = p.head.index
        val lastIndex: Int = p.last.index
        val firstLine: IndexedLine = indexedLines(firstIndex)
        val allLines: Vector[IndexedLine] = indexedLines.filter( il => {
            ( il.index > firstIndex) & ( il.index < lastIndex )
        })
        ( firstLine, allLines )
    })
    val lastSection: Vector[( IndexedLine, Vector[IndexedLine])] = {
        val firstIndex: Int = headingPairs.last.last.index
        val firstLine: IndexedLine = indexedLines(firstIndex)
        val allLines: Vector[IndexedLine] = indexedLines.filter( il => {
            ( il.index > firstIndex) 
        })
        val tup = ( firstLine, allLines )
        Vector[( IndexedLine, Vector[IndexedLine])](tup)
    }
    allButLast ++ lastSection
}

In [ ]:
def splitTitle( testString: String ): (String, String, String) = {
    val charVec = testString.toVector
    val filteredVec = charVec.filter( c => {
        val s = c.toString
        val rpl = s.replaceAll("[A-Z]", "")
        rpl == ""
    })
    val firstChar: Char = filteredVec.head.toChar
    val firstPorIndex: Int = charVec.indexOf(firstChar)
    val greekTitle: String = testString.take(firstPorIndex - 1)
    val porTitle: String = testString.takeRight( testString.size - firstPorIndex )
    val chapterId: String = {
        val rx = "^[0-9]+".r
        val foundOption: Option[String] = rx.findFirstIn(testString)
        foundOption.getOrElse("NO_ID")
        
    }
    (chapterId, greekTitle, porTitle)
}

We can make two CEX blocks, one for Greek and one for Portuguese. First we define our URN-base:

In [ ]:
val urnBase = CtsUrn("urn:cts:greekLit:tlg0096.tlg002:")

Greek first…

In [ ]:
val greekBlock: Vector[String] = mappedHeadings.map( h => {
    val heading: IndexedLine = h._1
    val section: IndexedLine = h._2.head
    val splitHeading = splitTitle(heading.text)
    val sectionId = splitHeading._1
    val sectionHeading = splitHeading._2
    val versionUrn = urnBase.addVersion("First1K-grc1")
    Vector(
        s"${versionUrn}${sectionId}.head#${sectionHeading}",
        s"${versionUrn}${sectionId}.text#${section.text}"
    )
}).flatten

Now Portugues…

In [ ]:
val portBlock: Vector[String] = mappedHeadings.map( h => {
    val heading: IndexedLine = h._1
    val section: IndexedLine = h._2.last
    val splitHeading = splitTitle(heading.text)
    val sectionId = splitHeading._1
    val sectionHeading = splitHeading._3
    val versionUrn = urnBase.addVersion("mcdezotti")
    Vector(
        s"${versionUrn}${sectionId}.head#${sectionHeading}",
        s"${versionUrn}${sectionId}.text#${section.text}"
    )
}).flatten

In [ ]:
portBlock.last